In [2]:
import pandas as pd
import matplotlib.pyplot as plt

/tmp/ipykernel_558268/1492400551.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df = pd.read_csv('data/HuluRaw.csv')
df

,index,show/id,show/cache_time,show/canonical_name,show/clips_count,show/description,show/episodes_count,show/feature_films_count,show/film_clips_count,show/genre,...,show/show_rollups/showtime/html5_games_count,show/show_rollups/showtime/html5_feature_films_count,show/show_rollups/showtime/html5_film_clips_count,show/show_rollups/showtime/html5_film_trailers_count,show/show_rollups/showtime/html5_videos_count,show/show_rollups/showtime/non_rating_videos_count,show/show_rollups/showtime/highest_rating,show/annotations/0,show/annotations/1,Unnamed: 181
0,0,54,2017-08-10T14:53:04+00:00,family-guy,288,The adventures of an endearingly ignorant dad ...,288,0,0,Animation and Cartoons,...,0,0,0,0,288,0,TV-MA,NaN,NaN,NaN
1,1,6979,2017-08-10T14:14:33+00:00,south-park,3874,"Underpants-stealing gnomes, a talking Christma...",274,0,0,Comedy,...,0,0,0,0,4148,0,TV-MA,NaN,NaN,NaN
2,2,53,2017-08-10T14:46:51+00:00,american-dad,121,This screwball family full of radically differ...,172,0,0,Comedy,...,0,0,0,0,121,0,TV-14,NaN,NaN,NaN
3,3,389,2017-08-10T15:00:39+00:00,law-and-order-special-victims-unit,176,This hard-hitting and emotional series from NB...,406,0,0,Drama,...,0,0,0,0,176,0,TV-14,NaN,NaN,NaN
4,4,1603,2017-08-10T14:54:31+00:00,naruto-shippuden,16,The Village Hidden in the Leaves is home to th...,597,0,0,Anime,...,0,0,0,0,612,1,TV-14,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,14114,2017-08-10T16:01:12+00:00,east-los-high,34,East Los High is a breakthrough Hulu Original ...,60,0,0,Drama,...,0,0,0,0,37,0,TV-14,NaN,NaN,NaN
996,996,1349,2017-08-10T15:46:19+00:00,sesame-street,7,"The beloved show ""Sesame Street"" combines educ...",60,0,0,Kids,...,0,0,0,0,67,0,TV-Y,NaN,NaN,NaN
997,997,23170,2017-08-10T15:34:53+00:00,casual,20,From the Academy Award Nominated Director of “...,36,0,0,Comedy,...,0,0,0,0,43,0,TV-MA,NaN,NaN,NaN
998,998,16720,2017-08-10T15:44:45+00:00,chicago-pd,66,District 21 of the Chicago Police Department i...,23,0,0,Drama,...,0,0,0,0,66,0,TV-14,https://ib.hulu.com/company_logo/2?bg=dim&colo...,NaN,NaN


In [4]:
columns = ['show/episodes_count', 'show/genre', 'show/name', 'show/rating', 'show/seasons_count', 'show/show_rollups/free/released_at']
rename = {k: k.split('/')[-1] for k in columns}

df = df[columns]
df = df.rename(columns=rename)
df

,episodes_count,genre,name,rating,seasons_count,released_at
0,288,Animation and Cartoons,Family Guy,4.353707,15,2007-09-05T19:01:09Z
1,274,Comedy,South Park,4.363032,20,2011-06-15T03:26:09Z
2,172,Comedy,American Dad!,4.114007,9,2007-09-21T23:03:15Z
3,406,Drama,Law & Order: Special Victims Unit,4.622856,18,2007-10-07T01:50:50Z
4,597,Anime,Naruto Shippuden,4.400626,9,2008-12-04T01:22:06Z
...,...,...,...,...,...,...
995,60,Drama,East Los High,4.156869,4,2013-04-29T06:00:10Z
996,60,Kids,Sesame Street,3.363742,3,2016-01-27T20:47:19Z
997,36,Comedy,Casual,4.536517,3,2015-10-01T00:37:16Z
998,23,Drama,Chicago P.D.,4.549656,1,2013-12-19T03:22:10Z


In [5]:
df.shape

(1000, 6)

In [7]:
df.isnull().sum()

episodes_count      0
genre               0
name                0
rating              0
seasons_count       0
released_at       284
dtype: int64